#Prepare environment

In [1]:
%matplotlib inline

In [2]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #pour faire des graph dans la bonne couleur etc.. par default
import glob
import re
import csv as csv
import random as rn
import math
import ast #http://stackoverflow.com/questions/1894269/convert-string-representation-of-list-to-list-in-python
#import to_datetime
import datetime
from ggplot import *
from __future__ import division

#from lxml.html import parse
from urllib2 import urlopen
import webbrowser

#Upload datasets

> Define products to be taken into account for credited conversions

>/!\ Which products do you want to take into account????

In [3]:
#On part donc sur de la performance avec un budget de 85K€ et un CPM à 2€
 
#Les objectifs sont les suivants :
#80% cible Part et 20% cible pros
#KPI 1 : ROI gamme Mobilité à 6€ : PD/DI/DN/TI/TN/AM/PR
#KPI 2 : CA réexpédition 50% des ventes : TI/TN/DI/DN
#KPI 3 : 50% du CA en conquête
 
#Pour la fenêtre d’attribution on sera sur du 15 J en PC et 48H en PV


#http://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe
#gamme_prod = ['DN', 'DI', 'PD', 'TI', 'TN']
#gamme_prod = ['DN', 'DI', 'PD', 'TI', 'TN','Mixte']
gamme_prod = ['PD','DI','DN','TI','TN','AM','PR','Mixte']


QC_cpm = 2.0

> Name of the files to be uploaded

> These should be hosted in the root folder

In [4]:
#file_quanti 
adServerFile = 'Report_Quantcast_mobilite_Standard_v1_M88Z5cUHryFJpRP_AA1S.xlsx'
#file_quali 
detailedFile = 'Report_Quantcast_mobilite_conversion_view_PNJjl0461vpIzpMgHrA7.csv'

In [5]:
#first file to be uploaded details of products part of each conversion
detailedConversions = pd.read_csv(detailedFile, dtype=object)

#We need to update the date format and strip hours
#http://stackoverflow.com/questions/26387986/strip-time-from-an-object-date-in-pandas
detailedConversions['conversion_date'] = pd.to_datetime(detailedConversions['Conversion date-hour'])
detailedConversions['conversion_date'] = detailedConversions['conversion_date'].apply(lambda x:x.date().strftime('%Y-%m-%d'))

#detailedConversions details products within each orders
#check what's in detailedConversions

#detailedConversions.count()

In [6]:
#then upload the second file. this is the regular 3PAS files with every conversions to which we contributed. 
xls_file = pd.ExcelFile(adServerFile)
adServerTable = xls_file.parse('DataView')

#adServerTable credits each order to a vendor
# /!\ We want to had details of orders containing products for which we are credited /!\
#check what's in adServerTable


#adServerTable.count()

#Create matching key on both datasets

In [7]:
#create function that will be used to create matching key by concatenating several columns
#http://stackoverflow.com/questions/23444858/concatenate-column-values-in-pandas-dataframe-with-nan-values
def concat(*args):
    strs = [str(arg) for arg in args if not pd.isnull(arg)]
    return '_'.join(strs) if strs else np.nan

np_concat = np.vectorize(concat)

In [8]:
#We need to define the key to join both file. 
#first 3PAS report Key
adServerTable['key'] = np_concat(adServerTable['Date'], adServerTable['Site/Offer ID'], adServerTable['Insertion ID'], adServerTable['Creative ID'])
#QA the created key
adServerTable[['Date','Site/Offer ID','Insertion ID','Creative ID', 'key']].head(2)

,Date,Site/Offer ID,Insertion ID,Creative ID,key
0,2016-03-08,1123,7490,1731,2016-03-08_1123_7490_1731
1,2016-03-08,1123,7483,1733,2016-03-08_1123_7483_1733


In [9]:
detailedConversions['key'] = np_concat(detailedConversions['conversion_date'], detailedConversions['Site-Offer ID'], detailedConversions['Insertion ID'], detailedConversions['Creative ID'])
#QA the created key
detailedConversions[['conversion_date','Site-Offer ID','Insertion ID','Creative ID', 'key']].head(2)

,conversion_date,Site-Offer ID,Insertion ID,Creative ID,key
0,2016-03-10,1123,7485,1731,2016-03-10_1123_7485_1731
1,2016-03-10,1123,7485,1731,2016-03-10_1123_7485_1731


#Define functions for latter use

> Let's define several functions that'll be used for cleaning datasets

In [10]:
# function aimed at cleaning revenue figures

def cleanrev(col):
    if col == '-':
        return 0.0
    else:
        return float(col)

In [11]:
# function aimed at cleaning products part of the orders

def cleanGamme(col,gamme_prod):
    items = list(set(col.split("|")) )
    if len(items) == 1:
        return items[0]
    elif re.search('|'.join(gamme_prod),col):
        return "Mixte"
    else:
        return "Mixte_non_comptabilise"
        

In [12]:
# function aimed at associating products and unit price
# /!\ We need to verify every product are added and quantity matches

def cleanProduitMix(df):
    items = df[0].split("|")
    prices = df[1].split("|")
    quantity = df[2].split("|")
    if len(items) == 1:
        return items[0] + '-%s-%s' %(float(prices[0]), float(quantity[0])) #*float(quantity[0]))
    else:
#        mix = '%s-' %len(items)
        l = len(items)
        mix = ''
        for i in range(0,len(items)-2):
            mix = mix + items[i] + '-%s-%s|' %(float(prices[i]), float(quantity[i])) #*float(quantity[i]))
        return mix + items[l-1] + '-%s-%s' %(float(prices[l-1]), float(quantity[l-1])) #*float(quantity[l-1]))

In [13]:
#function aimed at concatenating several columns separated by '-'
def concatenate(df):
    if len(df)>0:
        c = ''
        l = len(df)-1
        for i in range (0, l-1):
            c = c + df[i] + '-'
        return c + df[l]
    else:
        return ''

In [14]:
#function to be used when filtering out products not in required
def filter_products(x):
    if re.search('|'.join(gamme_prod),x):
        return "true"
    else:
        return "false"

#Treat and refine qualitative file

> The goal here is to agregate orders constituted by credited products to the 3PAS file

In [15]:
details3PAS = detailedConversions[['key','nom_site','gamme_produit', 'prix_produit','nombre_produit' ,'prix_ht_zanox']]
details3PAS = pd.DataFrame(details3PAS).dropna()
    
details3PAS['revenue_transaction'] = details3PAS.prix_ht_zanox.apply(lambda x: cleanrev(x))
details3PAS['panier'] = details3PAS.gamme_produit.apply(lambda x: cleanGamme(x,gamme_prod))

#details3PAS.head(4)
details3PAS[details3PAS.key == '2016-03-14_1123_7491_1740'].head(3)

,key,nom_site,gamme_produit,prix_produit,nombre_produit,prix_ht_zanox,revenue_transaction,panier
108,2016-03-14_1123_7491_1740,Part,VREL,4.92,1,4.92,4.92,VREL
130,2016-03-14_1123_7491_1740,Part,LRE,6.61,1,6.61,6.61,LRE
148,2016-03-14_1123_7491_1740,Part,LRE,5.33,1,5.33,5.33,LRE


In [16]:
dataSet3PAS= details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)
dataSet3PAS['selected'] = dataSet3PAS['panier'].apply(lambda x: filter_products(x))
dataSet3PAS = dataSet3PAS[dataSet3PAS['selected'] == 'true']

transactions = dataSet3PAS.groupby(['key', 'nom_site']).agg({
                                     'panier': 'count',
                                     'revenue_transaction': sum
                                               }).reset_index()    

transactions.columns = ['key', 'nom_site', 'relevant_conversions', 'total_revenue']
transactions.head(3)


#transactions = dataSet3PAS.groupby(['key']).count().reset_index()
#revenue = dataSet3PAS.groupby(['key']).sum().reset_index()
#transactions.head(10)
#revenue.head(10)
#ds = transactions.join(revenue).reset_index()
#ds.columns = ['key', 'revenue', 'conversions']
#ds.head(5)

,key,nom_site,relevant_conversions,total_revenue
0,2016-03-10_1123_7485_1731,Part,3,73.39
1,2016-03-10_1123_7487_1735,Part,1,58.50
2,2016-03-10_1123_7490_1731,Part,1,25.50


In [17]:
#reorder products with unit prices

#http://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe
#http://stackoverflow.com/questions/19914937/applying-function-with-multiple-arguments-to-create-a-new-pandas-column

details3PAS['order'] = details3PAS[['gamme_produit','prix_produit','nombre_produit']].apply(cleanProduitMix, axis = 1)
details3PAS = details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)
details3PAS.head(3)

,key,nom_site,revenue_transaction,panier,order
0,2016-03-10_1123_7485_1731,Part,37.50,PD,PD-37.5-1.0
1,2016-03-10_1123_7485_1731,Pro,17.00,MTEL,MTEL-17.0-1.0
2,2016-03-10_1123_7485_1731,Part,16.32,MTEL,MTEL-16.32-24.0


In [18]:
# It seems files are not accurately populated. 
# As basket details don't match actual basket (sum does not equals revenue transaction) we cannot use Mixed baskets

details3PAS[details3PAS.key == '2016-03-14_1123_7491_1740'].head(10)

,key,nom_site,revenue_transaction,panier,order
108,2016-03-14_1123_7491_1740,Part,4.92,VREL,VREL-4.92-1.0
130,2016-03-14_1123_7491_1740,Part,6.61,LRE,LRE-6.61-1.0
148,2016-03-14_1123_7491_1740,Part,5.33,LRE,LRE-5.33-1.0
164,2016-03-14_1123_7491_1740,Part,46.00,DN,DN-46.0-1.0
165,2016-03-14_1123_7491_1740,Part,12.50,AELC,AELC-12.5-1.0
187,2016-03-14_1123_7491_1740,Part,46.00,TN,TN-46.0-1.0
188,2016-03-14_1123_7491_1740,Part,42.30,Mixte,Produit_physique_part-8.4-1.0|DN-25.5-1.0


> We need to create a row per product purchased

In [19]:
#http://stackoverflow.com/questions/17116814/pandas-how-do-i-split-text-in-a-column-into-multiple-rows
#http://stackoverflow.com/questions/33622470/fast-way-to-split-column-into-multiple-rows-in-pandas

#create a column with only porduct and unit prices
s = details3PAS['order'].str.split("|", expand=True).stack() #.reset_index()
i = s.index.get_level_values(0)
df = details3PAS.loc[i].copy()
df["order"] = s.values
#df.head(20)

#make two columns out of this. might be tricky if we want to keep the original DF 
#df_bis = pd.DataFrame(df['order'].str.split("-", 1).tolist(), columns = ['product', 'price'], index = df.index) #.stack() #.reset_index()

df_bis = df 
df_bis["product"], df_bis["price"], df_bis["quantity"] = zip(*df_bis["order"].str.split('-',2))


df_bis["product"] = df_bis["product"].apply(lambda x: str(x))
df_bis["price"] = df_bis["price"].apply(lambda x: float(x))
df_bis["quantity"] = df_bis["quantity"].apply(lambda x: float(x))


df_bis[df_bis.key == '2016-03-14_1123_7491_1740'].head()

,key,nom_site,revenue_transaction,panier,order,product,price,quantity
108,2016-03-14_1123_7491_1740,Part,4.92,VREL,VREL-4.92-1.0,VREL,4.92,1
130,2016-03-14_1123_7491_1740,Part,6.61,LRE,LRE-6.61-1.0,LRE,6.61,1
148,2016-03-14_1123_7491_1740,Part,5.33,LRE,LRE-5.33-1.0,LRE,5.33,1
164,2016-03-14_1123_7491_1740,Part,46.00,DN,DN-46.0-1.0,DN,46.00,1
165,2016-03-14_1123_7491_1740,Part,12.50,AELC,AELC-12.5-1.0,AELC,12.50,1


In [20]:
#compute revenue for each product sold
df_bis['revenue_product'] = df_bis['price'] * df_bis['quantity']

df_bis = df_bis[['key', 'nom_site', 'product', 'revenue_product']]

df_bis[df_bis.key == '2016-03-14_1123_7491_1740'].head(10)

,key,nom_site,product,revenue_product
108,2016-03-14_1123_7491_1740,Part,VREL,4.92
130,2016-03-14_1123_7491_1740,Part,LRE,6.61
148,2016-03-14_1123_7491_1740,Part,LRE,5.33
164,2016-03-14_1123_7491_1740,Part,DN,46.00
165,2016-03-14_1123_7491_1740,Part,AELC,12.50
187,2016-03-14_1123_7491_1740,Part,TN,46.00
188,2016-03-14_1123_7491_1740,Part,Produit_physique_part,8.40
188,2016-03-14_1123_7491_1740,Part,DN,25.50


In [21]:
#keep only products concerned by our advertisign campaign

df_final = df_bis
#using our function defined above
df_final["product_bis"] = df_final["product"].apply(lambda x: filter_products(x))

#filtering out false products (products not concerned by the campaign)
df_final = df_final[df_final['product_bis'] == "true"]
df_final.head()

/private/var/localadmin/.conda/envs/py27/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,key,nom_site,product,revenue_product,product_bis
0,2016-03-10_1123_7485_1731,Part,PD,37.5,true
4,2016-03-10_1123_7487_1735,Part,PD,58.5,true
7,2016-03-10_1123_7490_1731,Part,DN,25.5,true
9,2016-03-10_1123_7492_1735,Part,DN,25.5,true
11,2016-03-10_1123_7485_1731,Part,MTAM,7.0,true


In [29]:
#Let's sum valid revenue per transaction key
# Group the data frame by month and item and extract a number of stats from each group

df_final_bis = df_final.groupby(['key', 'nom_site']).agg({
                                     'revenue_product':sum
                                               }).reset_index()    

#Add the total number conversions we stored in transactions DataFrame
#transactions.columns = ['key', 'nom_site', 'relevant_conversions', 'total_revenue']
#df_final_cleaned = pd.concat([df_final_bis,transactions], keys=['key'], join= 'inner' ,ignore_index=True)
df_final_cleaned = pd.merge(df_final_bis,transactions, how= 'inner', on = ['key', 'nom_site'] ,left_index=False,right_index=False)

#df_final_cleaned = df_final_bis.append(transactions)

df_final_cleaned.head(3)

,key,nom_site,revenue_product,relevant_conversions,total_revenue
0,2016-03-10_1123_7485_1731,Part,70.0,3,73.39
1,2016-03-10_1123_7487_1735,Part,58.5,1,58.50
2,2016-03-10_1123_7490_1731,Part,25.5,1,25.50


> Performances summary

# /!\ filter dataset - first option not considered anymore since we computed revenue per product per transaction key

In [29]:
#details3PAS.count()
details3PAS.head(4)

,key,nom_site,revenue_transaction,panier,order
0,2016-03-10_1123_7485_1731,Part,37.50,PD,PD-37.5-1.0
1,2016-03-10_1123_7485_1731,Pro,17.00,MTEL,MTEL-17.0-1.0
2,2016-03-10_1123_7485_1731,Part,16.32,MTEL,MTEL-16.32-24.0
3,2016-03-10_1123_7492_1735,Part,48.21,Mixte_non_comptabilise,Produit_physique_part-3.55-1.0|Produit_physiqu...


In [44]:
dataSet3PAS= details3PAS.drop(['gamme_produit','nombre_produit','prix_produit','prix_ht_zanox'],1)
dataSet3PAS.head(3)

ValueError: labels ['gamme_produit' 'nombre_produit' 'prix_produit' 'prix_ht_zanox'] not contained in axis

In [17]:
dataSet3PAS = dataSet3PAS[dataSet3PAS.panier.isin(gamme_prod)]
dataSet3PAS.head(5)

,key,nom_site,revenue_transaction,panier
0,2016-03-10_1123_7485_1731,Part,37.5,PD
4,2016-03-10_1123_7487_1735,Part,58.5,PD
7,2016-03-10_1123_7490_1731,Part,25.5,DN
9,2016-03-10_1123_7492_1735,Part,25.5,DN
13,2016-03-10_1123_7485_1731,Part,25.5,DN


In [18]:
transactions = dataSet3PAS.groupby(['key']).count()
revenue = dataSet3PAS.groupby(['key']).sum()
ds = pd.DataFrame(transactions,revenue)

In [19]:
transactions = pd.DataFrame(dataSet3PAS.groupby(["key"]).sum()["revenue_transaction"])
revenue = pd.DataFrame(dataSet3PAS.groupby(["key"]).count()["panier"])
ds = transactions.join(revenue).reset_index()
#c = np.append('key', 'revenue', 'conversions',axis =0)
ds.columns = ['key', 'revenue', 'conversions']
ds.head(5)

,key,revenue,conversions
0,2016-03-10_1123_7485_1731,63.0,2
1,2016-03-10_1123_7487_1735,58.5,1
2,2016-03-10_1123_7490_1731,25.5,1
3,2016-03-10_1123_7492_1735,25.5,1
4,2016-03-11_1123_7484_1733,37.5,1


In [56]:
dataSet3PAS[dataSet3PAS.key == '2016-03-10_1123_7485_1731']

,key,nom_site,revenue_transaction,panier
0,2016-03-10_1123_7485_1731,Part,37.5,PD
13,2016-03-10_1123_7485_1731,Part,25.5,DN


In [57]:
ds[ds.key == '2016-03-10_1123_7485_1731']

,key,revenue,conversions
0,2016-03-10_1123_7485_1731,63,2


# Load quantitative 3PAS report

> load quantitative 3PAS report and merge two files

In [42]:
#final_ds = pd.merge(adServerTable,ds, on='key')
#conversions are missing is it needed
final_ds = pd.merge(adServerTable,df_final_cleaned, on='key')

In [43]:
final_ds[final_ds.key == '2016-03-10_1123_7485_1731'].head(5)

,Date,Site/Offer ID,Site/Offer,Insertion ID,Insertion,Creative ID,Creative,Creative Sizes,Imp.,Clicks,...,PI conv. BNUM_Confirmation achat_Part,Revenues (PC) BNUM_Confirmation achat_Pro,Revenues (PC) BNUM_Confirmation achat_Part,Revenues (PI) BNUM_Confirmation achat_Pro,Revenues (PI) BNUM_Confirmation achat_Part,key,nom_site,revenue_product,relevant_conversions,total_revenue
3,2016-03-10,1123,Quantcast,7485,Conquête pack déménagement Particuliers_300x250,1731,300x250_LaPoste_Demenagement,300x250,17539,2,...,5,-,37.5,17,67.05,2016-03-10_1123_7485_1731,Part,70,3,73.39


In [44]:
#final_ds = final_ds[['Date', 'Site/Offer ID','Site/Offer', 'Insertion ID', 'Insertion', 'Creative ID', 'Creative', 'Creative Sizes','Imp.', 'Clicks','conversions', 'revenue', 'key' ]]
#revenue_product	relevant_conversions

final_ds = final_ds[['Date', 'Site/Offer ID','Site/Offer', 'Insertion ID', 'Insertion', 'Creative ID', 'Creative', 'Creative Sizes','Imp.', 'Clicks','relevant_conversions', 'revenue_product','total_revenue', 'key' ]]

In [34]:
final_ds.head(4)

,Date,Site/Offer ID,Site/Offer,Insertion ID,Insertion,Creative ID,Creative,Creative Sizes,Imp.,Clicks,relevant_conversions,revenue_product,key
0,2016-03-10,1123,Quantcast,7492,Retargeting pack déménagement Particuliers_728x90,1735,728x90_LaPoste_Demenagement,728x90,30698,7,2,25.5,2016-03-10_1123_7492_1735
1,2016-03-10,1123,Quantcast,7487,Conquête pack déménagement Particuliers_728x90,1735,728x90_LaPoste_Demenagement,728x90,22671,3,1,58.5,2016-03-10_1123_7487_1735
2,2016-03-10,1123,Quantcast,7490,Retargeting pack déménagement Particuliers_300...,1731,300x250_LaPoste_Demenagement,300x250,8422,3,1,25.5,2016-03-10_1123_7490_1731
3,2016-03-10,1123,Quantcast,7485,Conquête pack déménagement Particuliers_300x250,1731,300x250_LaPoste_Demenagement,300x250,17539,2,3,70.0,2016-03-10_1123_7485_1731


In [45]:
final_ds['cost'] = QC_cpm * final_ds['Imp.'] / 1000

In [46]:
def test(ds1):
    a = ds1[0]
    b = ds1[1]
    return b/a


In [47]:
final_ds['roi'] = final_ds['revenue_product'] / final_ds['cost']

In [48]:
final_ds.head(4)

,Date,Site/Offer ID,Site/Offer,Insertion ID,Insertion,Creative ID,Creative,Creative Sizes,Imp.,Clicks,relevant_conversions,revenue_product,total_revenue,key,cost,roi
0,2016-03-10,1123,Quantcast,7492,Retargeting pack déménagement Particuliers_728x90,1735,728x90_LaPoste_Demenagement,728x90,30698,7,2,25.5,73.71,2016-03-10_1123_7492_1735,61.396,0.415337
1,2016-03-10,1123,Quantcast,7487,Conquête pack déménagement Particuliers_728x90,1735,728x90_LaPoste_Demenagement,728x90,22671,3,1,58.5,58.50,2016-03-10_1123_7487_1735,45.342,1.290195
2,2016-03-10,1123,Quantcast,7490,Retargeting pack déménagement Particuliers_300...,1731,300x250_LaPoste_Demenagement,300x250,8422,3,1,25.5,25.50,2016-03-10_1123_7490_1731,16.844,1.513892
3,2016-03-10,1123,Quantcast,7485,Conquête pack déménagement Particuliers_300x250,1731,300x250_LaPoste_Demenagement,300x250,17539,2,3,70.0,73.39,2016-03-10_1123_7485_1731,35.078,1.995553


In [49]:
final_ds.to_csv('3pas_report_final.csv', index=False,encoding='utf-8')
print('Done')

Done
